In [1]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis
# !pip install tensorflow numpy pandas
# !pip install spectral

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras as keras
from keras import layers as layers
import numpy as np

In [4]:
import os, timeit
import cv2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from math import inf as inf
import numpy as np

In [5]:
from spectral.io import envi as envi
from spectral import imshow

In [6]:
from sklearn.decomposition import IncrementalPCA

In [7]:
import keras_tuner as kt
import sys

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [10]:
#Testing Constants
TESTING = False

#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
IMAGE_COUNT = int(1200/4)
NUM_VARIETIES = 8
NUM_OF_BANDS = 15
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30
NUM_LAYERS_EACH_BLOCK = [1, 1]
NUM_CHAN_PER_BLOCK = [[64,128,128],[128,128,256]]

ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001

In [11]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [12]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [13]:
def getROI(img,band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [14]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [15]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img    

In [16]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [17]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [18]:
data_augmentation = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img,band_number):
    new_img = getUsefulImage(img,band_number)
    augmented_image = data_augmentation(new_img) 
    return augmented_image

def checkAugumentedImage(augmented_image):
    aug_band = augmented_image[:,:,0]
    filled_area_ratio = (np.count_nonzero(aug_band))/(aug_band.shape[0]*aug_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [19]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 10
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

In [20]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [21]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [22]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            images.append(img)
            images_label.append(v)
        except:
            pass

In [23]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [24]:
i = 0

In [25]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        aug_img = getAugumentedImage(img,BAND_NUMBER)
        
        if checkAugumentedImage(aug_img):
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1 
        
        i+=1
        print(i,count)    
    
    if TESTING:
        break
        
toc = end_timer()
show_time(tic,toc)

Testing started
1 1
2 1
3 2
4 2
5 2
6 2
7 3
8 3
9 3
10 3
11 3
12 3
13 4
14 5
15 6
16 7
17 7
18 7
19 8
20 8
21 9
22 9
23 10
24 10
25 11
26 11
27 12
28 13
29 14
30 14
31 14
32 15
33 15
34 16
35 16
36 16
37 17
38 17
39 18
40 19
41 20
42 21
43 22
44 23
45 24
46 24
47 24
48 25
49 25
50 25
51 25
52 26
53 26
54 27
55 27
56 28
57 28
58 29
59 29
60 30
61 30
62 31
63 32
64 32
65 32
66 32
67 32
68 33
69 34
70 34
71 34
72 34
73 34
74 34
75 35
76 35
77 36
78 36
79 37
80 37
81 38
82 39
83 39
84 40
85 40
86 41
87 42
88 43
89 43
90 43
91 43
92 44
93 44
94 45
95 45
96 46
97 47
98 47
99 47
100 48
101 48
102 48
103 48
104 49
105 49
106 49
107 50
108 50
109 50
110 50
111 50
112 50
113 50
114 50
115 50
116 51
117 51
118 51
119 51
120 52
121 52
122 52
123 52
124 52
125 52
126 53
127 54
128 54
129 55
130 55
131 55
132 55
133 55
134 56
135 56
136 57
137 58
138 59
139 60
140 61
141 61
142 62
143 62
144 62
145 63
146 64
147 64
148 64
149 64
150 64
151 64
152 64
153 65
154 65
155 65
156 66
157 67
158 67
159 68
1

1096 137
1097 137
1098 137
1099 138
1100 138
1101 138
1102 139
1103 139
1104 140
1105 140
1106 140
1107 140
1108 140
1109 141
1110 141
1111 141
1112 142
1113 143
1114 143
1115 143
1116 143
1117 144
1118 144
1119 144
1120 145
1121 145
1122 145
1123 145
1124 146
1125 146
1126 146
1127 147
1128 148
1129 148
1130 149
1131 149
1132 150
1133 151
1134 151
1135 151
1136 151
1137 151
1138 152
1139 152
1140 152
1141 152
1142 153
1143 154
1144 154
1145 154
1146 154
1147 154
1148 155
1149 155
1150 155
1151 156
1152 156
1153 157
1154 157
1155 157
1156 158
1157 158
1158 159
1159 160
1160 160
1161 160
1162 161
1163 162
1164 162
1165 163
1166 164
1167 165
1168 165
1169 165
1170 166
1171 166
1172 166
1173 166
1174 167
1175 168
1176 168
1177 168
1178 168
1179 169
1180 170
1181 170
1182 170
1183 170
1184 170
1185 170
1186 170
1187 171
1188 171
1189 172
1190 172
1191 173
1192 173
1193 173
1194 173
1195 173
1196 173
1197 173
1198 174
1199 175
1200 175
1201 175
1202 175
1203 176
1204 177
1205 177
1206 177
1

2045 174
2046 175
2047 176
2048 177
2049 177
2050 177
2051 177
2052 177
2053 177
2054 177
2055 178
2056 178
2057 178
2058 178
2059 178
2060 178
2061 178
2062 178
2063 178
2064 178
2065 178
2066 179
2067 180
2068 181
2069 181
2070 182
2071 183
2072 183
2073 183
2074 184
2075 184
2076 184
2077 185
2078 185
2079 185
2080 186
2081 187
2082 188
2083 188
2084 188
2085 188
2086 189
2087 189
2088 189
2089 189
2090 189
2091 190
2092 190
2093 190
2094 190
2095 191
2096 191
2097 191
2098 191
2099 191
2100 192
2101 193
2102 193
2103 194
2104 194
2105 195
2106 196
2107 196
2108 197
2109 197
2110 197
2111 197
2112 197
2113 198
2114 198
2115 198
2116 198
2117 199
2118 199
2119 199
2120 199
2121 200
2122 201
2123 201
2124 201
2125 202
2126 202
2127 202
2128 203
2129 203
2130 204
2131 205
2132 206
2133 207
2134 207
2135 207
2136 208
2137 208
2138 209
2139 209
2140 209
2141 209
2142 209
2143 209
2144 210
2145 210
2146 211
2147 211
2148 212
2149 213
2150 213
2151 213
2152 213
2153 213
2154 213
2155 213
2

2982 23
2983 24
2984 24
2985 24
2986 24
2987 24
2988 24
2989 24
2990 25
2991 26
2992 27
2993 28
2994 29
2995 30
2996 31
2997 31
2998 31
2999 32
3000 33
3001 33
3002 33
3003 34
3004 35
3005 35
3006 36
3007 37
3008 38
3009 39
3010 39
3011 40
3012 40
3013 40
3014 40
3015 40
3016 41
3017 42
3018 42
3019 42
3020 43
3021 43
3022 43
3023 43
3024 44
3025 45
3026 45
3027 46
3028 46
3029 47
3030 48
3031 49
3032 49
3033 49
3034 50
3035 51
3036 52
3037 52
3038 53
3039 54
3040 55
3041 55
3042 55
3043 55
3044 55
3045 55
3046 55
3047 56
3048 57
3049 58
3050 58
3051 58
3052 59
3053 60
3054 60
3055 60
3056 61
3057 62
3058 63
3059 63
3060 63
3061 64
3062 64
3063 64
3064 64
3065 65
3066 65
3067 65
3068 65
3069 65
3070 65
3071 65
3072 65
3073 66
3074 67
3075 68
3076 69
3077 69
3078 69
3079 70
3080 71
3081 71
3082 71
3083 71
3084 71
3085 71
3086 71
3087 72
3088 73
3089 73
3090 73
3091 73
3092 74
3093 74
3094 75
3095 76
3096 76
3097 77
3098 77
3099 77
3100 77
3101 78
3102 78
3103 78
3104 79
3105 80
3106 81


3945 120
3946 120
3947 120
3948 121
3949 121
3950 121
3951 121
3952 122
3953 123
3954 123
3955 124
3956 125
3957 126
3958 126
3959 126
3960 126
3961 126
3962 126
3963 126
3964 127
3965 127
3966 127
3967 128
3968 128
3969 128
3970 128
3971 128
3972 128
3973 128
3974 128
3975 128
3976 128
3977 129
3978 129
3979 129
3980 130
3981 131
3982 131
3983 131
3984 132
3985 133
3986 133
3987 134
3988 134
3989 134
3990 134
3991 134
3992 134
3993 135
3994 136
3995 137
3996 137
3997 137
3998 137
3999 138
4000 139
4001 140
4002 140
4003 140
4004 140
4005 141
4006 141
4007 141
4008 141
4009 142
4010 142
4011 142
4012 143
4013 143
4014 143
4015 143
4016 143
4017 144
4018 145
4019 146
4020 146
4021 146
4022 147
4023 147
4024 147
4025 147
4026 147
4027 148
4028 149
4029 150
4030 150
4031 151
4032 151
4033 151
4034 152
4035 152
4036 152
4037 152
4038 153
4039 153
4040 154
4041 155
4042 156
4043 156
4044 156
4045 157
4046 158
4047 159
4048 160
4049 161
4050 161
4051 161
4052 162
4053 162
4054 163
4055 163
4

...

22423 276
22424 276
22425 276
22426 276
22427 276
22428 276
22429 276
22430 277
22431 278
22432 278
22433 278
22434 279
22435 280
22436 280
22437 281
22438 281
22439 281
22440 281
22441 282
22442 283
22443 283
22444 283
22445 283
22446 283
22447 283
22448 284
22449 284
22450 285
22451 285
22452 286
22453 287
22454 287
22455 287
22456 288
22457 288
22458 288
22459 288
22460 288
22461 289
22462 290
22463 290
22464 290
22465 291
22466 292
22467 293
22468 294
22469 295
22470 295
22471 295
22472 295
22473 295
22474 295
22475 296
22476 297
22477 297
22478 297
22479 297
22480 298
22481 298
22482 299
22483 299
22484 299
22485 299
22486 299
22487 300
22488 1
22489 2
22490 3
22491 3
22492 3
22493 4
22494 5
22495 5
22496 6
22497 7
22498 8
22499 8
22500 8
22501 9
22502 10
22503 10
22504 11
22505 11
22506 12
22507 12
22508 12
22509 12
22510 13
22511 13
22512 13
22513 14
22514 15
22515 16
22516 16
22517 17
22518 17
22519 17
22520 17
22521 18
22522 18
22523 18
22524 19
22525 20
22526 21
22527 21
2252

23282 55
23283 55
23284 55
23285 55
23286 55
23287 55
23288 56
23289 57
23290 57
23291 57
23292 57
23293 57
23294 58
23295 58
23296 59
23297 59
23298 59
23299 60
23300 60
23301 60
23302 60
23303 60
23304 60
23305 60
23306 61
23307 62
23308 63
23309 63
23310 63
23311 64
23312 65
23313 66
23314 67
23315 68
23316 68
23317 69
23318 69
23319 70
23320 70
23321 71
23322 71
23323 71
23324 71
23325 71
23326 72
23327 73
23328 73
23329 73
23330 74
23331 74
23332 74
23333 74
23334 74
23335 74
23336 74
23337 75
23338 76
23339 76
23340 77
23341 78
23342 79
23343 80
23344 81
23345 81
23346 81
23347 81
23348 81
23349 81
23350 81
23351 81
23352 81
23353 82
23354 83
23355 83
23356 83
23357 84
23358 84
23359 84
23360 84
23361 84
23362 84
23363 84
23364 84
23365 85
23366 85
23367 85
23368 85
23369 86
23370 86
23371 86
23372 87
23373 87
23374 88
23375 89
23376 90
23377 90
23378 91
23379 92
23380 92
23381 92
23382 93
23383 93
23384 93
23385 93
23386 93
23387 94
23388 95
23389 95
23390 95
23391 96
23392 96
2

24134 59
24135 59
24136 60
24137 61
24138 61
24139 61
24140 62
24141 63
24142 63
24143 63
24144 63
24145 63
24146 64
24147 65
24148 65
24149 65
24150 66
24151 66
24152 67
24153 67
24154 68
24155 68
24156 68
24157 68
24158 69
24159 70
24160 70
24161 71
24162 72
24163 72
24164 72
24165 72
24166 72
24167 72
24168 73
24169 74
24170 74
24171 74
24172 74
24173 75
24174 75
24175 76
24176 76
24177 76
24178 77
24179 78
24180 78
24181 78
24182 79
24183 79
24184 80
24185 80
24186 80
24187 81
24188 82
24189 82
24190 83
24191 84
24192 84
24193 85
24194 86
24195 86
24196 86
24197 87
24198 88
24199 88
24200 88
24201 89
24202 89
24203 90
24204 90
24205 91
24206 92
24207 92
24208 93
24209 94
24210 94
24211 95
24212 95
24213 95
24214 95
24215 95
24216 96
24217 96
24218 96
24219 96
24220 97
24221 97
24222 97
24223 97
24224 98
24225 99
24226 99
24227 99
24228 100
24229 101
24230 102
24231 102
24232 102
24233 102
24234 102
24235 102
24236 102
24237 103
24238 103
24239 104
24240 105
24241 105
24242 105
2424

24997 52
24998 52
24999 53
25000 53
25001 53
25002 53
25003 53
25004 53
25005 54
25006 54
25007 54
25008 54
25009 54
25010 54
25011 54
25012 54
25013 54
25014 55
25015 55
25016 55
25017 55
25018 56
25019 57
25020 57
25021 57
25022 57
25023 57
25024 57
25025 57
25026 57
25027 57
25028 57
25029 57
25030 57
25031 57
25032 57
25033 57
25034 57
25035 57
25036 57
25037 58
25038 58
25039 59
25040 59
25041 59
25042 59
25043 59
25044 60
25045 60
25046 61
25047 61
25048 61
25049 62
25050 62
25051 62
25052 62
25053 62
25054 63
25055 63
25056 63
25057 64
25058 64
25059 64
25060 65
25061 65
25062 65
25063 65
25064 65
25065 65
25066 65
25067 65
25068 65
25069 65
25070 65
25071 66
25072 66
25073 66
25074 67
25075 67
25076 67
25077 67
25078 67
25079 67
25080 67
25081 67
25082 67
25083 67
25084 67
25085 67
25086 68
25087 68
25088 68
25089 68
25090 69
25091 69
25092 69
25093 69
25094 69
25095 70
25096 70
25097 70
25098 70
25099 70
25100 70
25101 70
25102 71
25103 71
25104 71
25105 71
25106 71
25107 71
2

25845 226
25846 226
25847 226
25848 226
25849 226
25850 226
25851 226
25852 226
25853 226
25854 226
25855 226
25856 226
25857 226
25858 226
25859 227
25860 228
25861 229
25862 229
25863 229
25864 229
25865 230
25866 231
25867 231
25868 231
25869 231
25870 231
25871 231
25872 231
25873 231
25874 231
25875 232
25876 232
25877 232
25878 232
25879 232
25880 233
25881 233
25882 234
25883 234
25884 235
25885 235
25886 235
25887 235
25888 235
25889 235
25890 235
25891 235
25892 236
25893 236
25894 236
25895 237
25896 238
25897 238
25898 239
25899 239
25900 239
25901 239
25902 239
25903 239
25904 239
25905 240
25906 240
25907 240
25908 241
25909 241
25910 241
25911 241
25912 241
25913 241
25914 241
25915 241
25916 242
25917 243
25918 244
25919 244
25920 244
25921 244
25922 244
25923 245
25924 245
25925 246
25926 246
25927 246
25928 246
25929 247
25930 247
25931 247
25932 248
25933 248
25934 248
25935 249
25936 249
25937 249
25938 249
25939 249
25940 249
25941 249
25942 249
25943 249
25944 250


In [26]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [27]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
# import math, sys, pdb, os
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Conv2DTranspose, Flatten, Dense, Conv1D, AveragePooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers import concatenate
from keras.models import Model

import os, pdb, timeit
import numpy as np
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.cm as cm
import cv2
from keras import activations
import vis
# from vis.visualization import visualize_saliency, overlay
# from vis.utils import utils

In [28]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:]) # L2-norm by default        
        
    return data

In [29]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.clim(0,sum(cm[0,:]))
    plt.xlabel('Predicted label')

In [30]:
def top_K_classification_accuracy(y_predicted, y_true, K=1):

    num_samples = y_predicted.shape[0]
    num_classes = y_predicted.shape[1]

    if K > num_classes:
        sys.exit(1)

    temp = np.zeros((num_samples,))

    for idx in range(num_samples):
        curr_predicted = np.argsort(y_predicted[idx,:])
        curr_predicted = curr_predicted[::-1] # descending

        if y_true[idx] in curr_predicted[:K]:
            temp[idx] = 1

    return 100.0 * np.sum(temp)/num_samples

In [31]:
def single_conv_block(x, kernel_size=3, activation_type='relu', num_filters=128,use_bias=False):
    # Batch norm
    x = BatchNormalization()(x)
    x = Conv2D(num_filters, kernel_size, activation=None, use_bias=use_bias, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)
    return x

In [32]:
def conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D=[32,64,128]):

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[0])

    x = single_conv_block(x,kernel_size,activation_type,num_filters_first_conv1D[1],use_bias=True)

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[2])

    # Dropout
    return Dropout(dropout_rate)(x)

In [33]:
def createBlock_ResNet2D(x, num_layers, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D):

    for idx_layer in range(num_layers):
        x = conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D)

    return x

In [34]:
def ResNet2D_classifier(data_num_rows, data_num_cols, num_classes, kernel_size=3, num_layers_each_block=[],
                        num_chan_per_block=[], activation_type='relu', dropout_rate=0.0, num_input_chans=1, num_nodes_fc=64):

    input_data = Input(shape=(data_num_rows, data_num_cols, num_input_chans))

    # Input layer: Conv2D -> activation
    x = Conv2D(num_chan_per_block[0][0], kernel_size, activation=None, use_bias=True, padding='same',
               kernel_initializer='truncated_normal')(input_data)

    # Activation
    x = Activation(activation_type)(x)


    #  Blocks & Downsampling Layers
    for idx_block in range(len(num_layers_each_block)):
        x = createBlock_ResNet2D(x, num_layers_each_block[idx_block], kernel_size, activation_type, dropout_rate,
                                 num_chan_per_block[idx_block])

        x = BatchNormalization()(x)

        if idx_block != len(num_layers_each_block)-1:
            x = Conv2D(num_chan_per_block[idx_block][1]*2, kernel_size, strides = 2, activation=None, use_bias=True, padding='valid',
                   kernel_initializer='truncated_normal')(x)
        else:
            x = GlobalAveragePooling2D()(x)

        x = Dropout(dropout_rate)(x)

    # Output layer
    x = BatchNormalization()(x)
    
    x = Flatten()(x)

    x = Dense(units=num_nodes_fc, activation=None, kernel_initializer='truncated_normal')(x)
#     x = Dense(units=32, activation=None, kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)
    output_data = Dense(units=num_classes, activation='softmax', kernel_initializer='truncated_normal')(x)

    return Model(inputs=input_data, outputs=output_data)

In [35]:
def evaluate(model,dataset,dataset_label,normalization_type):
    print("--------------Make Predictions--------------")    
    x = np.array(dataset)
    labels = np.array(dataset_label)
    
    # Normalize the data
    x = normalizeDataWholeSeed(x,normalization_type=normalization_type)
    
    num = x.shape[0]

    tic = start_timer()
    labels_predicted = model.predict(x)
    toc = end_timer()
    show_time(tic,toc)
    
    print(labels_predicted)
    print("--------")
    # Classification accuracy
    labels_integer_format = labels
    labels_predicted_integer_format = np.argmax(labels_predicted, axis=1)

    acc_top2 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=2)
    acc_top1 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=1)
    
    # Confusion matrices
    confusion_matrix_results = confusion_matrix(labels_integer_format, labels_predicted_integer_format)
    print("Confusion matrix = ")
    print(confusion_matrix_results)
    print("------------------------------------------------")

In [36]:
def predict(model,normalization_type):
    evaluate(model,train_dataset,train_dataset_label,normalization_type)
    
    evaluate(model,test_dataset,test_dataset_label,normalization_type)
    


In [37]:
def createAndTrainResNetB():
    
    learning_rate_base = LEARNING_RATE_BASE
    kernel_size = 3
    dropout_rate = 0.15 
    activation_type = ACTIVATION_TYPE
    num_nodes_fc = 256
    wheat_types =  VARIETIES
    normalization_type = 'max'
    num_layers_each_block = NUM_LAYERS_EACH_BLOCK
    num_chan_per_block = NUM_CHAN_PER_BLOCK
    N_classes = len(wheat_types)
    
    ############ Load data ############
    print("--------------Load Data--------------")

    # Load training data and their corresponding labels
    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training,normalization_type=normalization_type)
    
    # Extract some information
    num_training = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_bands = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = ResNet2D_classifier(data_num_rows = N_spatial[0], 
                                data_num_cols = N_spatial[1], 
                                num_classes = N_classes,
                                kernel_size = kernel_size, 
                                num_layers_each_block = num_layers_each_block,
                                num_chan_per_block = num_chan_per_block, 
                                activation_type = activation_type,
                                dropout_rate = dropout_rate, 
                                num_input_chans = N_bands, 
                                num_nodes_fc = num_nodes_fc)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    print("---------Completed---------")
    return model

In [38]:
model = createAndTrainResNetB()

--------------Load Data--------------
--------------Done--------------
--------------Create a model--------------
---------Completed---------


In [40]:
for x in range(2):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    print(model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.20,shuffle=True))
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  1  to  20
Testing started
Epoch 1/20
384/384 - 145s - loss: 1.3781 - accuracy: 0.5513 - val_loss: 3.7821 - val_accuracy: 0.0085 - 145s/epoch - 378ms/step
Epoch 2/20
384/384 - 146s - loss: 1.3478 - accuracy: 0.5550 - val_loss: 3.6955 - val_accuracy: 0.0088 - 146s/epoch - 381ms/step
Epoch 3/20
384/384 - 147s - loss: 1.3175 - accuracy: 0.5685 - val_loss: 3.6187 - val_accuracy: 0.0098 - 147s/epoch - 382ms/step
Epoch 4/20
384/384 - 146s - loss: 1.2911 - accuracy: 0.5641 - val_loss: 3.5527 - val_accuracy: 0.0094 - 146s/epoch - 381ms/step
Epoch 5/20
384/384 - 146s - loss: 1.2846 - accuracy: 0.5591 - val_loss: 3.6047 - val_accuracy: 0.0104 - 146s/epoch - 380ms/step
Epoch 6/20
384/384 - 146s - loss: 1.2780 - accuracy: 0.5632 - val_loss: 3.5424 - val_accuracy: 0.0104 - 146s/epoch - 380ms/step
Epoch 7/20
384/384 - 162s - loss: 1.2683 - accuracy: 0.5649 - val_loss: 3.4949 - val_accuracy: 0.0101 - 162s/epoch - 423ms/step
Epoch 8/20
384/384 - 151s - loss: 1.2560 - accuracy: 0.5698 - va